# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [11]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
file_path = r'C:\Users\ogulc\Desktop\Sandbox_DSI\scaling_to_production\05_src\data\adult\adult.data'
adult_dt = pd.read_csv(file_path, header=None, names=columns)
adult_dt['income'] = (adult_dt['income'].str.strip() == '>50K').astype(int)




# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [12]:
X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
- The random state in the splitting function (random_state=42) is like setting a seed for a random number generator

+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?
- makes the experiment more reliable and comparable.

*(Comment here.)*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier

numeric_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])


numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])


X_preprocessed = preprocessor.fit_transform(X)


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor), 
    ('classifier', RandomForestClassifier())  
])


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [18]:

from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss

scoring = {
    'neg_log_loss': make_scorer(log_loss, greater_is_better=False, needs_proba=True),
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}


cv_results = cross_validate(pipeline, X, Y, cv=5, scoring=scoring, return_train_score=True)


print("Training Scores:")
print(f"Negative Log Loss: {-cv_results['train_neg_log_loss'].mean():.4f}") 
print(f"ROC AUC: {cv_results['train_roc_auc'].mean():.4f}")
print(f"Accuracy: {cv_results['train_accuracy'].mean():.4f}")
print(f"Balanced Accuracy: {cv_results['train_balanced_accuracy'].mean():.4f}")
print("\nValidation Scores:")
print(f"Negative Log Loss: {-cv_results['test_neg_log_loss'].mean():.4f}")  
print(f"ROC AUC: {cv_results['test_roc_auc'].mean():.4f}")
print(f"Accuracy: {cv_results['test_accuracy'].mean():.4f}")
print(f"Balanced Accuracy: {cv_results['test_balanced_accuracy'].mean():.4f}")



c:\Users\ogulc\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\metrics\_scorer.py:548: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
c:\Users\ogulc\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\ogulc\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:241: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Training Scores:
Negative Log Loss: 0.0807
ROC AUC: 1.0000
Accuracy: 0.9999
Balanced Accuracy: 0.9999

Validation Scores:
Negative Log Loss: 0.3680
ROC AUC: 0.9025
Accuracy: 0.8545
Balanced Accuracy: 0.7753


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [20]:

fold_results = {
    'Train Negative Log Loss': -cv_results['train_neg_log_loss'],  # Take negative and mean for train
    'Test Negative Log Loss': -cv_results['test_neg_log_loss']  # Take negative and mean for test
}
results_df = pd.DataFrame(fold_results)
results_df_sorted = results_df.sort_values(by='Test Negative Log Loss')
print("Fold-level results sorted by Test Negative Log Loss:")
print(results_df_sorted)


Fold-level results sorted by Test Negative Log Loss:
   Train Negative Log Loss  Test Negative Log Loss
3                 0.080816                0.355726
4                 0.081031                0.358866
0                 0.080355                0.368900
2                 0.080606                0.370531
1                 0.080472                0.385982


Calculate the mean of each metric. 

In [22]:
mean_metrics = results_df_sorted.mean()
print("Mean metrics across folds:")
print(mean_metrics)

Mean metrics across folds:
Train Negative Log Loss    0.080656
Test Negative Log Loss     0.368001
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [36]:
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score, balanced_accuracy_score

# Fit the model
pipeline.fit(X_train, Y_train)

# Predict the probabilities
y_pred_proba = pipeline.predict_proba(X_test)
y_pred_proba_last = pipeline.predict_proba(X_test)[:, 1]

# Evaluate all metrics
log_loss_score = log_loss(Y_test, y_pred_proba)
accuracy = accuracy_score(Y_test, y_pred_proba_last > 0.5)
balanced_accuracy = balanced_accuracy_score(Y_test, y_pred_proba_last > 0.5)
roc_auc = roc_auc_score(Y_test, y_pred_proba_last)

# Print metrics
print(f"Log loss: {log_loss_score}")
print(f"Accuracy: {accuracy}")
print(f"Balanced accuracy: {balanced_accuracy}")
print(f"ROC AUC: {roc_auc}")


Log loss: 0.38098118804430325
Accuracy: 0.8525949431876344
Balanced accuracy: 0.7715493479459297
ROC AUC: 0.9000330418118043


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Recoding the target variable income right after loading simplifies its format for machine learning models to ensure accurate performance evaluation and compatibility with standard metrics.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.